In [1]:
from sklearn.dummy import DummyClassifier
import pandas as pd
import numpy as np
import xgboost as xgb

# from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import (
    Normalizer,
    FunctionTransformer,
    RobustScaler,
    LabelEncoder,
    MinMaxScaler
)
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, StratifiedShuffleSplit
from sklearn.feature_selection import (
    SelectFromModel,
    VarianceThreshold,
    chi2,
    SelectKBest,
    mutual_info_classif,
)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kstest, shapiro, probplot, ks_2samp
import category_encoders as ce
from catboost import CatBoostClassifier
from sklearn.tree import export_graphviz
from dython.model_utils import random_forest_feature_importance
import mlflow
import os
import logging
import yaml
import warnings

/Users/wolfrieder/miniconda3/envs/code/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [3]:
warnings.filterwarnings("ignore")

with open("../params.yaml", "rb") as f:
    params = yaml.load(f, yaml.FullLoader)

mlflow.set_tracking_uri(params["ml_flow"]["MLFLOW_TRACKING_URI"])
os.environ["MLFLOW_TRACKING_USERNAME"] = params["ml_flow"][
    "MLFLOW_TRACKING_USERNAME"
]
os.environ["MLFLOW_TRACKING_PASSWORD"] = params["ml_flow"][
    "MLFLOW_TRACKING_PASSWORD"
]
mlflow.set_experiment("all_binary_experiments")
mlflow.sklearn.autolog()

In [4]:
train_data = pd.read_parquet(
    "../data/processed/chrome/08_12_2022/train_set_all_binary.parquet.gzip"
)
test_data = pd.read_parquet(
    "../data/processed/chrome/08_12_2022/test_set_all_binary.parquet.gzip"
)

In [5]:
train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)

In [6]:
# %%time
# dummy_clf = DummyClassifier(strategy="most_frequent", random_state=10)
# dummy_clf.fit(X_train, y_train)
# dummy_clf.score(X_train, y_train)

CPU times: user 26.1 ms, sys: 1.84 ms, total: 27.9 ms
Wall time: 26.6 ms


0.7157431508360171

In [7]:
# model = RandomForestClassifier(
#     n_estimators=100,
#     n_jobs=-1,
#     random_state=10,
#     criterion="log_loss",
#     max_features=None,
# )

# model = KNeighborsClassifier(n_jobs=-1)
# model = DecisionTreeClassifier()
# model = GradientBoostingClassifier()
# model = HistGradientBoostingClassifier()
# model = xgb.XGBClassifier()
# model = lgb.LGBMClassifier(class_weight="balanced")
# model = CatBoostClassifier(thread_count=-1)
# model = MLPClassifier()

In [6]:
def calculate_metrics(data, y_true):
    y_pred = model.predict(data)
    pred_probs = model.predict_proba(data)
    score = metrics.log_loss(y_true, pred_probs)
    auc_score = metrics.roc_auc_score(y_true, y_pred)
    f1_score = metrics.f1_score(y_true, y_pred)
    bal_acc = metrics.balanced_accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)
    mcc = metrics.matthews_corrcoef(y_true, y_pred)

    print("Model accuracy score : {0:0.4f}".format(metrics.accuracy_score(y_test, y_pred)))
    print("Model log-loss score : {0:0.4f}".format(score))
    print("Model auc score : {0:0.4f}".format(auc_score))
    print("Balanced accuracy score : {0:0.4f}".format(bal_acc))
    print("F1 score : {0:0.4f}".format(f1_score))
    print("Precision score : {0:0.4f}".format(precision))
    print("Recall score : {0:0.4f}".format(recall))
    print("Matthews correlation coefficient score : {0:0.4f}".format(mcc))
    print(metrics.classification_report(y_test, y_pred))

    disp_1 = metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
    disp_2 = metrics.PrecisionRecallDisplay.from_estimator(
        model, X_test, y_test, name="KNN"
    )
    disp_3 = metrics.RocCurveDisplay.from_estimator(model, X_test, y_test)
    mlflow.log_figure(disp_1.figure_, "cm.png")
    mlflow.log_figure(disp_2.figure_, "prec_recall.png")
    mlflow.log_figure(disp_3.figure_, "roc.png")

In [ ]:
X_train, y_train = train_data.iloc[:, :-1], train_data[["tracker"]]
X_test, y_test = test_data.iloc[:, :-1], test_data[["tracker"]]

del train_data
del test_data

In [ ]:
with mlflow.start_run():
    model = xgb.XGBClassifier()

    model.fit(X_train, y_train["tracker"].to_numpy())
    calculate_metrics(X_test, y_test)
mlflow.end_run()

[20:13:48] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
